In [67]:
import components
import pandas as pd
import numpy as np
path_results = "results/"

### simulation

#### initialize helper objects for simulation
- SimulationBroker
- VehicleGenerator
- ResultWriter

In [2]:
# simulation broker
path_Sim = "test_data/beam1/beam1-0.csv"
dtype_Sim = {
       'time': 'int64', 'type': 'category', 'vehicle': 'int64', 'parkingTaz': 'category','chargingPointType': 'category', 
       'primaryFuelLevel': 'float64', 'mode': 'category', 'currentTourMode': 'category', 'vehicleType': 'category', 
       'arrivalTime': 'float64', 'departureTime': 'float64', 'linkTravelTime': 'string', 'primaryFuelType': 'category', 
       'parkingZoneId': 'category','duration': 'float64' 
        }
SimBroker = components.SimBroker(path_Sim, dtype_Sim)

# vehicle generator
path_DataBase = "test_data/vehicleFiles/vehicletypes-Base_2035_20210204_updated.csv"
VehicleGenerator = components.VehicleGenerator(path_Sim, dtype_Sim, path_DataBase)

path_save = path_results+"sim1"
ResultWriter = components.ResultWriter(path_save)

In [3]:
# show head of vehicles dataframe
VehicleGenerator.vehicles.head(3)

,vehicleType
vehicle,
2517,ev-L1-0-to-50000-LowTech-2035-Midsize-BEV_300_XFC
9368,ev-L1-0-to-50000-LowTech-2035-Midsize-BEV_300_XFC
11104,ev-L1-0-to-50000-LowTech-2035-Car-BEV_300_XFC


#### create charging stations
1) map parkingZoneIds to charging stations with a dictionary
2) create charging Stations

In [74]:
# load infrastructure file into dataframe
path_infrastructure = "test_data/beam1/gemini-base-scenario-3-charging-no-household-infra16.csv"
usecols_infrastructure = ["taz", "parkingType", "chargingPointType", "parkingZoneId"]
dtype_infrastructure = {"taz": "int64", "parkingType": "category", "chargingPointType": "category", "parkingZoneId": "string"}
infrastructure = pd.read_csv(path_infrastructure, dtype=dtype_infrastructure, usecols=usecols_infrastructure)
infrastructure = infrastructure.set_index("parkingZoneId")
# filter infrastructure for only public fast and extreme fast charging
infrastructure = infrastructure.loc[infrastructure["parkingType"] == "Public"]
infrastructure = infrastructure.loc[infrastructure["chargingPointType"].str.contains("publicfc|publicxfc")]
infrastructure = infrastructure.sort_values(by = ["taz", "parkingZoneId"])

#now, make a dict of every parkingZoneId that belongs to a charging station
# here, we will split the chargingstation
chargingStationMappedToParkingZoneId = {}
chargingStationMappedToTaz = {}
stepsize = 200
i = 0 
j = 1
stop = False
while i < len(infrastructure) -1 :
    name = "chargingStation-" + str(j)
    # we want to make sure, that all the chargingBays of one TAZ are in one chargingStation
    if i + stepsize < len(infrastructure) :
        i_end = i+stepsize
        while infrastructure.iloc[i_end]["taz"] == infrastructure.iloc[i_end+1]["taz"]:
            i_end +=1 # if the taz is the same, we should increase reading to that taz
            if i_end +1 >= len(infrastructure) - 1 : # make sure, that we don't try to read in the next step something that doesnt exist
                stop = True
                break
    else: 
        i_end = len(infrastructure)-1
    if not stop:
        i_end += 1 # to also catch the last element
    slice = infrastructure.iloc[i:i_end]
    chargingStationMappedToParkingZoneId[name] = slice.index.to_list()
    chargingStationMappedToTaz[name] = list(set(slice["taz"].to_list())) # this removes duplicates
    i = i_end # start reading next cycle at i
    j += 1

# we convert chargingStationMappedToTaz to a dataframe to use search methods
chargingStationMappedToTaz = pd.DataFrame.from_dict(chargingStationMappedToTaz, orient='index')
chargingStationMappedToTaz = chargingStationMappedToTaz.transpose()

# #here, we will make chargingStation depending on TAZ. Therefore, get a list of the different TAZ and sort it:
# taz = infrastructure.taz.drop_duplicates().to_list()
# taz = sorted(taz)
# while i < len(taz):
#     name = "chargingStation-" + str(j)
#     # slice = infrastructure.loc[infrastructure["taz"].isin(taz[i:i+stepsize])]
#     chargingStationMappedToParkingZoneId[name] = slice.index.to_list()
#     i += stepsize
#     j += 1

In [33]:
list(set(slice["taz"].to_list()))

59

In [5]:
infrastructure.head(3)

,taz,parkingType,chargingPointType
parkingZoneId,,,
X-PEV-9-1,9,Public,publicfc(150.0|DC)
X-PEV-11-1,11,Public,publicfc(150.0|DC)
X-PEV-12-1,12,Public,publicfc(150.0|DC)


In [6]:
print(chargingStationMappedToParkingZoneId.keys())

dict_keys(['chargingStation-1', 'chargingStation-2', 'chargingStation-3'])


In [7]:
#create chargingStations
chargingStations = [] # list of charging stations
for i in chargingStationMappedToParkingZoneId:
    #chargingStations.append(i)

    ChargingStationId = i
    ChBaNum = len(chargingStationMappedToParkingZoneId[i])
    # make a list with the powers of the bays:
    ChBaMaxPower = []
    for j in chargingStationMappedToParkingZoneId[i]:
        power_string = infrastructure.loc[j, "chargingPointType"]
        ChBaMaxPower.append(components.chargingCapFromString(power_string))
    ChBaParkingZoneId = chargingStationMappedToParkingZoneId[i]
    calcBtmsGridProp = True

    # think about variable power limits
    # include interfaces like in diagram

    
    container = components.ChaDepLimCon(ChargingStationId=ChargingStationId, ResultWriter=ResultWriter, SimBroker = SimBroker, ChBaMaxPower=ChBaMaxPower, ChBaParkingZoneId=ChBaParkingZoneId, calcBtmsGridProp = True)
    chargingStations.append(container)
    print(ChargingStationId + " was created with " + str(container.ChBaNum) + " charging bays and " + str(container.BtmsSize) + "kWh BTM-Storage")

chargingStation-1 was created with 200 charging bays and 16600.0kWh BTM-Storage
chargingStation-2 was created with 200 charging bays and 18300.0kWh BTM-Storage
chargingStation-3 was created with 108 charging bays and 9300.0kWh BTM-Storage


- initialize helper objects for simulation input

In [8]:
PhySimDummy = components.PhySimDummy(chargingStations)
DermsDummy  = components.DermsDummy(chargingStations)

In [9]:
print(PhySimDummy.data['chargingStation-1'])

[nan, nan, 0.5, 16600.0]


#### initialize simulation
initialize Grid Constraints and actual time of charging Station

In [10]:
for x in chargingStations:
    GridPowerLower, GridPowerUpper = DermsDummy.output(x.ChargingStationId)
    x.initialize(t_start = SimBroker.t_act, GridPowerLower = GridPowerLower, GridPowerUpper = GridPowerUpper)

#### run simulation

In [39]:
'''Simulation settings:'''
timestep = 5 * 60


In [75]:
SimBroker.reset()
while not SimBroker.eol():
    # Sim Broker Step
    slice = SimBroker.step(timestep)
    # generate Vehicles if charging Plug in event
    for i in range(0, len(slice)):
        if slice.iloc[i]["type"] == "ChargingPlugInEvent":
            # generate vehicle
            vehicle = VehicleGenerator.generateVehicleSO(slice.iloc[i])
            # let vehicle arrive at a charging station - here depending on taz
            taz = int(slice.iloc[i]["parkingTaz"]) # this is a str
            #find out which chargingStation belongs to the taz
            res = chargingStationMappedToTaz.isin([taz]).any().values
            index = np.where(res == True)[0][0]
            #let vehicles arrive at designated chargingStation
            chargingStations[index].arrival(vehicle)
    #control action
    # call step function
        # specify step function of controller
        # add vehicle releases
        # add ResultWriter Routines


0
0
0
0
2
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
2
0
2
2
0
0
0
0
0
0
2
0
0
0
0
0
0
2
0
0
0
0
0
0
0
2
0
0
2
0
0
2
0
0
0
0
0
0
0
0
0
2
2
0
0
0
2
0
0
0
0
0
2
0
2
2
0
0
0
0
2
0
0
0
0
2
2
0
0
2
0
2
2
0
0
0
2
0
0
0
2
0
0
2
0
0
0
0
2
0
0
2
0
0
0
0
0
0
0
2
0
0
2
2
2
0
0
0
2
2
0
0
0
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
2
0
0
2
0
0
0
0
2
0
2
2
0
2
0
0
0
2
0
0
2
0
0
0
0
0
2
2
0
0
0
2
0
0
0
2
2
0
2
0
0
0
2
2
0
2
0
0
0
0
0
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
2
2
2
2
0
2
2
0
0
0
0
0
0
0
2
0
0
2
0
0
0
0
1
2
0
0
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
0
2
2
0
0
2
0
2
2
2
0
0
0
2
0
0
0
2
0
0
0
0
2
0
0
0
0
0
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
2
0
1
2
2
2
0
0
0
0
2
2
0
0
2
2
0
0
2
2
2
2
2
0
2
0
2
2
0
2
0
2
1
0
2
2
0
2
2
0
0
0
2
0
0
0
0
0
0
0
2
0
2
2
2
0
2
0
0
0
0
0
2
2
0
0
0
2
2
2
2
2
0
0
2
2
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
1
1
2
0
0
0
2
0
0
0
1
2
2
1
2
1
1
0
2
2


In [73]:
res = df1.isin([518]).any().values
index = np.where(res == True)[0][0]

0

In [31]:
chargingStationMappedToParkingZoneId

{'chargingStation-1': ['X-PEV-9-1',
  'X-PEV-11-1',
  'X-PEV-12-1',
  'X-PEV-21-1',
  'X-PEV-39-1',
  'X-PEV-69-1',
  'X-PEV-75-1',
  'X-PEV-88-1',
  'X-PEV-98-1',
  'X-PEV-103-1',
  'X-PEV-104-1',
  'X-PEV-110-1',
  'X-PEV-114-1',
  'X-PEV-132-1',
  'X-PEV-142-1',
  'X-PEV-187-1',
  'X-PEV-188-1',
  'X-PEV-192-1',
  'X-PEV-199-1',
  'X-PEV-203-1',
  'X-PEV-212-1',
  'X-PEV-225-1',
  'X-PEV-234-1',
  'X-PEV-235-1',
  'X-PEV-241-1',
  'X-PEV-248-1',
  'X-PEV-257-1',
  'X-PEV-278-1',
  'X-PEV-295-1',
  'X-PEV-309-1',
  'X-PEV-315-1',
  'X-PEV-325-1',
  'X-PEV-330-1',
  'X-PEV-331-1',
  'X-PEV-355-1',
  'X-PEV-356-1',
  'X-PEV-371-1',
  'X-PEV-387-1',
  'X-PEV-389-1',
  'X-PEV-401-1',
  'X-PEV-402-1',
  'X-PEV-412-1',
  'X-PEV-415-1',
  'X-PEV-416-1',
  'X-PEV-421-1',
  'X-PEV-430-1',
  'X-PEV-439-1',
  'X-PEV-449-1',
  'X-PEV-460-1',
  'X-PEV-462-1',
  'X-PEV-465-1',
  'X-PEV-471-1',
  'X-PEV-475-1',
  'X-PEV-478-1',
  'X-PEV-485-1',
  'X-PEV-486-1',
  'X-PEV-497-1',
  'X-PEV-504-1',
  '

#### save results

In [ ]:
ResultWriter.save()